In [1]:
import warnings
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display, Markdown, HTML, Javascript
from tqdm import tqdm


def alert(message='Loop completed!'):
    # Check if the system supports 'say' and notifications
    os.system(f'osascript -e \'display notification "{message}" with title "Notification"\'')
    # Speak the alert
    os.system(f'say "{message}"')        
        
InteractiveShell.ast_node_interactivity = "all"
warnings.filterwarnings("ignore")
tqdm.pandas()

In [82]:
import pandas as pd

df = pd.read_csv('data/raw/turnout_20241112.csv')

In [5]:
import json
import pandas as pd
import requests
import time

from datetime import datetime
from pprint import pprint
from typing import List, Dict, Optional

In [6]:
class MeasuresFetcher:
    def __init__(self, api_token: str):
        """
        Initialize the MeasuresFetcher with your API token.
        
        Args:
            api_token (str): Your CivicEngine OAuth token
        """
        self.api_url = "https://bpi.civicengine.com/graphql"
        self.headers = {
            "Content-Type": "application/json",
            "Accept": "application/json",
            "Authorization": f"Bearer {api_token}"
        }

    def fetch_measures(self, 
                      limit: int = 10, 
                      state: Optional[str] = None,
                      after_date: Optional[str] = None,
                      latitude: Optional[float] = None,
                      longitude: Optional[float] = None) -> Dict:
        """
        Fetch measures from the API.
        
        Args:
            limit (int): Number of elections to fetch (default: 10)
            state (str, optional): Filter by state code (e.g., 'CA')
            after_date (str, optional): Filter elections after this date (YYYY-MM-DD)
            latitude (float, optional): Latitude for location filtering
            longitude (float, optional): Longitude for location filtering
            
        Returns:
            dict: Raw API response
        """
        # Build location parameter if coordinates are provided
        location_param = ""
        if latitude is not None and longitude is not None:
            location_param = 'location: { point: { latitude: %f, longitude: %f } }' % (latitude, longitude)

        # Build the GraphQL query
        query = """
        {
          elections(
            filterBy: { electionDay: { eq: "2024-11-05" } }
            first: %d
            %s
          ) {
            nodes {
              id
              name
              electionDay
              state
              measures {
                nodes {
                  id
                  name
                  title
                  summary
                  text
                  conSnippet
                  proSnippet
                  state
                  arguments {
                    text
                    proCon
                    sourceUrl
                  }
                }
              }
            }
          }
        }
        """ % (limit, location_param)

        # Make the API request
        try:
            response = requests.post(
                self.api_url,
                headers=self.headers,
                json={"query": query}
            )
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            raise Exception(f"API request failed: {str(e)}")

    def process_measures(self, response: Dict) -> List[Dict]:
        """
        Process the API response and extract measures data.
        
        Args:
            response (dict): Raw API response
            
        Returns:
            list: List of processed measure dictionaries
        """
        processed_measures = []
        
        try:
            elections = response["data"]["elections"]["nodes"]
            
            for election in elections:
                election_date = election["electionDay"]
                election_name = election["name"]
                election_state = election["state"]
                
                for measure in election["measures"]["nodes"]:
                    processed_measure = {
                        "id": measure["id"],
                        "name": measure["name"],
                        "title": measure["title"],
                        "summary": measure["summary"],
                        "text": measure["text"],
                        "pro_snippet": measure["proSnippet"],
                        "con_snippet": measure["conSnippet"],
                        "state": measure["state"],
                        "election_name": election_name,
                        "election_date": election_date,
                        "arguments": measure["arguments"]
                    }
                    processed_measures.append(processed_measure)
                    
        except KeyError as e:
            raise Exception(f"Error processing API response: {str(e)}")
            
        return processed_measures


In [92]:
# Example usage
api_token = "weEtK6jhOCg8lgw2nRTPleyr17LsOvBfRz04drpWfWk"
fetcher = MeasuresFetcher(api_token)

# Fetch measures
response = fetcher.fetch_measures(
    limit=5,  # Fetch from 5 elections
    state=None,  # Optional state filter
    after_date=None  # Optional date filter
)

# Process measures
measures_by_location = fetch_measures_for_locations(results_df.sample(5, random_state=42), fetcher)
for location_key, measures in measures_by_location.items():
        print('\n\n', '===' * 25, '\n\n')
        print(f"\nLocation: {location_key}")
        print(f"Found {len(measures)} measures")
        for measure in measures[:5]:  # Print first 5 measures as example
            print('\n', '#' * 25, '\n')
            print(f"- {measure['name']}: {measure['election_date']}")
            pprint(measure)

Exception: API request failed: 401 Client Error: Unauthorized for url: https://bpi.civicengine.com/graphql

9

In [53]:
import yaml

with open('data/raw/ballotready.txt', 'r') as f:
    ballotready_data = f.read()
    
def safe_eval(y):
    ret_val = None
    try:
        ret_val =  {'measures': int(y.removesuffix(' measures').removeprefix('Found '))}
    except Exception as e:
        try:
            ret_val =  eval(y)
        except Exception as e:
            try:
                ret_val =  yaml.load(y, yaml.Loader)
            except Exception as e:
                try:
                    y1, _, y2 = y.partition('\n')
                    ret_val = {
                        **yaml.load(y1, yaml.Loader),
                        'measures': int(y2.removesuffix(' measures').removeprefix('Found '))
                    }
                except Exception as e:
                    raise
                    print(y)
                    print(e)
                    
#     if 'Location' in y:
#         print(ret_val)
#         print(y)
        
    return ret_val
                
                
    
data = [
    safe_eval(z.strip())
    for x in ballotready_data[75:].split('#########################')
    for y in x.strip().partition('\n')
    for z in y.split(' ===========================================================================')
]
data = [x for y in data for x in ([y] if isinstance(y, dict) else (y if y is not None else []))]

In [64]:
rows = []
row = {}
for point in data:
    first_key = list(point.keys())[0]
    if 'Location' in point:
        if len(row):
            rows.append(row)
        row = point.copy()
    elif 'arguments' in point:
        row['data'] = row.get('data', []) + [point]
    elif ('Amendment' in first_key) or ('Question' in first_key) or ('Referendum' in first_key):
        row['amendments'] = row.get('amendments', []) + [first_key]
        row['amendment_dates'] = row.get('amendment_dates', []) + [point[first_key]]
    else:
        row.update(point)
rows.append(row)

measure_df = pd.DataFrame(rows).dropna()

In [65]:
measure_df

,Location,measures,amendments,amendment_dates,data
0,41092_Grant_KY-CD04_Kentucky,55,"[Amendment 1, Amendment 2, Local Referendum 1,...","[2024-11-05, 2024-11-05, 2024-11-05, 2024-11-0...","[{'arguments': [{'proCon': 'CON', 'sourceUrl':..."
1,67036_Harper_KS-CD04_Kansas,11,"[Question 1, Question 1, Question 1, Question ...","[2024-11-05, 2024-11-05, 2024-11-05, 2024-11-0...","[{'arguments': [], 'con_snippet': None, 'elect..."
2,40513_Fayette_KY-CD06_Kentucky,55,"[Amendment 1, Amendment 2, Local Referendum 1,...","[2024-11-05, 2024-11-05, 2024-11-05, 2024-11-0...","[{'arguments': [{'proCon': 'CON', 'sourceUrl':..."
4,62650_Morgan_IL-CD15_Illinois,100,"[Advisory Question 2, Advisory Question 3, Adv...","[2024-11-05, 2024-11-05, 2024-11-05, 2024-11-0...","[{'arguments': [], 'con_snippet': 'A ""no"" vote..."


In [85]:
def get_row(row):
    zip_code, county, district, state = row.Location.split('_')
    row['zip'], row['County'], row['District'] = \
        int(zip_code), county, district
    row['data'] = [
        {**a, 'amendment_name': n, 'amendment_date': d}
        for a, n, d in zip(row['data'], row['amendments'], row['amendment_dates'])
    ]
    return row

def update_row(row):
    for k, v in row.data.items():
        row[k] = v
    return row


analysis_df = measure_df.apply(
    get_row,
    axis=1
).explode('data').apply(update_row, axis=1).set_index(['County', 'District', 'zip']).join(
    df.set_index(['County', 'district', 'zip']).drop(columns=['measures'])
)

In [86]:
analysis_df

Location  measures  \
County  District zip   district                                             
Grant   KY-CD04  41092 KY-CD04     41092_Grant_KY-CD04_Kentucky        55   
                       KY-CD04     41092_Grant_KY-CD04_Kentucky        55   
                       KY-CD04     41092_Grant_KY-CD04_Kentucky        55   
                       KY-CD04     41092_Grant_KY-CD04_Kentucky        55   
                       KY-CD04     41092_Grant_KY-CD04_Kentucky        55   
Harper  KS-CD04  67036 KS-CD04      67036_Harper_KS-CD04_Kansas        11   
                       KS-CD04      67036_Harper_KS-CD04_Kansas        11   
                       KS-CD04      67036_Harper_KS-CD04_Kansas        11   
                       KS-CD04      67036_Harper_KS-CD04_Kansas        11   
                       KS-CD04      67036_Harper_KS-CD04_Kansas        11   
Fayette KY-CD06  40513 KY-CD06   40513_Fayette_KY-CD06_Kentucky        55   
                       KY-CD06   40513_Fayette_KY-CD06_Kentucky        55   
                       KY-CD06   40513_Fayette_KY-CD06_Kentucky        55   
                       KY-CD06   40513_Fayette_KY-CD06_Kentucky        55   
                       KY-CD06   40513_Fayette_KY-CD06_Kentucky        55   
Morgan  IL-CD15  62650 IL-CD15    62650_Morgan_IL-CD15_Illinois       100   
                       IL-CD15    62650_Morgan_IL-CD15_Illinois       100   
                       IL-CD15    62650_Morgan_IL-CD15_Illinois       100   
                       IL-CD15    62650_Morgan_IL-CD15_Illinois       100   
                       IL-CD15    62650_Morgan_IL-CD15_Illinois       100   

                                                                        amendments  \
County  District zip   district                                                      
Grant   KY-CD04  41092 KY-CD04   [Amendment 1, Amendment 2, Local Referendum 1,...   
                       KY-CD04   [Amendment 1, Amendment 2, Local Referendum 1,...   
                       KY-CD04   [Amendment 1, Amendment 2, Local Referendum 1,...   
                       KY-CD04   [Amendment 1, Amendment 2, Local Referendum 1,...   
                       KY-CD04   [Amendment 1, Amendment 2, Local Referendum 1,...   
Harper  KS-CD04  67036 KS-CD04   [Question 1, Question 1, Question 1, Question ...   
                       KS-CD04   [Question 1, Question 1, Question 1, Question ...   
                       KS-CD04   [Question 1, Question 1, Question 1, Question ...   
                       KS-CD04   [Question 1, Question 1, Question 1, Question ...   
                       KS-CD04   [Question 1, Question 1, Question 1, Question ...   
Fayette KY-CD06  40513 KY-CD06   [Amendment 1, Amendment 2, Local Referendum 1,...   
                       KY-CD06   [Amendment 1, Amendment 2, Local Referendum 1,...   
                       KY-CD06   [Amendment 1, Amendment 2, Local Referendum 1,...   
                       KY-CD06   [Amendment 1, Amendment 2, Local Referendum 1,...   
                       KY-CD06   [Amendment 1, Amendment 2, Local Referendum 1,...   
Morgan  IL-CD15  62650 IL-CD15   [Advisory Question 2, Advisory Question 3, Adv...   
                       IL-CD15   [Advisory Question 2, Advisory Question 3, Adv...   
                       IL-CD15   [Advisory Question 2, Advisory Question 3, Adv...   
                       IL-CD15   [Advisory Question 2, Advisory Question 3, Adv...   
                       IL-CD15   [Advisory Question 2, Advisory Question 3, Adv...   

                                                                   amendment_dates  \
County  District zip   district                                                      
Grant   KY-CD04  41092 KY-CD04   [2024-11-05, 2024-11-05, 2024-11-05, 2024-11-0...   
                       KY-CD04   [2024-11-05, 2024-11-05, 2024-11-05, 2024-11-0...   
                       KY-CD04   [2024-11-05, 2024-11-05, 2024-11-05, 2024-11-0...   
                       KY-CD04   [202

In [90]:
analysis_df['text_len'] = analysis_df.text.apply(len)

In [91]:
analysis_df[['text_len', 'complexity']]

text_len  complexity
County  District zip   district                      
Grant   KY-CD04  41092 KY-CD04       3753    0.318540
                       KY-CD04        699    0.318540
                       KY-CD04        506    0.318540
                       KY-CD04        152    0.318540
                       KY-CD04        140    0.318540
Harper  KS-CD04  67036 KS-CD04        185    0.289170
                       KS-CD04        535    0.289170
                       KS-CD04        350    0.289170
                       KS-CD04        382    0.289170
                       KS-CD04        197    0.289170
Fayette KY-CD06  40513 KY-CD06       3753    0.285806
                       KY-CD06        699    0.285806
                       KY-CD06        506    0.285806
                       KY-CD06        152    0.285806
                       KY-CD06        140    0.285806
Morgan  IL-CD15  62650 IL-CD15        175    0.263037
                       IL-CD15        268    0.263037
                       IL-CD15        215    0.263037
                       IL-CD15        121    0.263037
                       IL-CD15        369    0.263037